In [1]:

%pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 148.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the necessary packages and modules
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score



pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [5]:
#target variable
data_2 = pd.read_csv(r'final_data_3.csv')
data_2['Target'] = data_2['Open'].shift(-1)
data_2 = data_2.drop([], axis=1)
data_2['Date'] = pd.to_datetime(data_2['Date'], format="%d-%m-%Y")
data_2 = data_2[data_2.Date < '2024-11-01']
data_2 = data_2[data_2.Date > '2021-12-31']
data_2 = data_2.dropna()

# data_2['Final_sentiment_Max_Mean'] = data_2['Final_sentiment_Max_Mean'] * -1
# data_2['Final_sentiment_Max_Min'] = data_2['Final_sentiment_Max_Min'] * -1
# data_2['Final_sentiment_Max_Max'] = data_2['Final_sentiment_Max_Max'] * -1

In [6]:
data_2

,Date,Close,High,Low,Open,Volume,Month,Year,SMA,EWMA_100,MiddleBand,UpperBand,LowerBand,RSI,ForceIndex,EMV,Final_sentiment_Max_Max,Final_sentiment_Max_Min,Final_sentiment_Max_Mean,India_Holidays,Exchange_holidays_SETTLEMENT_HOLIDAY,Exchange_holidays_TRADING_HOLIDAY,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Target
214,2022-01-01,1330.250000,1340.800049,1311.050049,1325.099976,235438,1,2022,1370.832009,1292.662534,1370.832009,1446.166932,1295.497086,40.737013,0.000000e+00,-74998.46341,0,0,0,0,False,False,False,False,True,False,False,False,False,1325.099976
215,2022-01-02,1330.250000,1340.800049,1311.050049,1325.099976,235438,1,2022,1371.591008,1293.416871,1371.591008,1444.419502,1298.762514,40.737013,0.000000e+00,-74998.46341,0,0,0,0,False,False,False,False,False,True,False,False,False,1340.000000
216,2022-01-03,1346.900024,1356.199951,1336.900024,1340.000000,130549,1,2022,1372.683008,1294.489931,1372.683008,1442.221919,1303.144097,45.047519,2.173644e+06,-16845.61786,0,0,0,0,False,False,False,True,False,False,False,False,False,1357.000000
217,2022-01-04,1366.699951,1375.000000,1340.000000,1357.000000,192988,1,2022,1373.916006,1295.938338,1373.916006,1440.691538,1307.140474,49.730108,3.821148e+06,-10850.02117,0,0,0,0,False,False,False,False,False,False,False,True,False,1366.699951
218,2022-01-05,1399.650024,1408.300049,1366.699951,1366.699951,394705,1,2022,1375.622007,1298.018080,1375.622007,1440.518162,1310.725852,56.389936,1.300556e+07,11451.93671,0,0,0,0,False,False,False,False,False,False,False,False,True,1400.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2024-10-26,1644.000000,1718.900024,1625.000000,1700.000000,707626,10,2024,1842.327991,1821.857720,1842.327991,2071.794812,1612.861169,23.819444,0.000000e+00,-86222.08729,0,0,0,0,False,False,False,False,True,False,False,False,False,1700.000000
1244,2024-10-27,1644.000000,1718.900024,1625.000000,1700.000000,707626,10,2024,1837.941990,1818.335785,1837.941990,2074.059625,1601.824354,23.819444,0.000000e+00,-86222.08729,0,0,0,0,False,False,False,False,False,True,False,False,False,1644.000000
1245,2024-10-28,1613.150024,1661.699951,1588.550049,1644.000000,850381,10,2024,1832.938989,1814.272701,1832.938989,2077.319628,1588.558351,20.775807,-2.623423e+07,-113247.66300,-1,-1,-1,0,False,False,False,True,False,False,False,False,False,1612.000000
1246,2024-10-29,1635.099976,1641.000000,1588.000000,1612.000000,540217,10,2024,1828.325989,1810.724726,1828.325989,2078.810084,1577.841893,27.840868,1.185774e+07,-107880.12390,0,0,0,0,False,False,False,False,False,False,False,True,False,1615.000000


In [7]:
print(data_2.Date.max(), data_2.Date.min())

2024-10-30 00:00:00 2022-01-01 00:00:00


In [8]:
data_2.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Month', 'Year',
       'SMA', 'EWMA_100', 'MiddleBand', 'UpperBand', 'LowerBand', 'RSI',
       'ForceIndex', 'EMV', 'Final_sentiment_Max_Max',
       'Final_sentiment_Max_Min', 'Final_sentiment_Max_Mean', 'India_Holidays',
       'Exchange_holidays_SETTLEMENT_HOLIDAY',
       'Exchange_holidays_TRADING_HOLIDAY', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Target'],
      dtype='object')

In [9]:
#xgboost

In [13]:
#HYPERPARAMETERE TUNING

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 500],  # Number of boosting rounds (trees)
    'max_depth': [3, 6, 10, 20, 30],  # Maximum depth of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Step size for boosting adjustments
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features considered per tree split
}

# Initialize the Random Forest model
rf = XGBRegressor(random_state=42)

# Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, 
                           n_jobs=-1, verbose=0)
grid_search.fit(data_2.drop(['Target', 'Date'], axis = 1), data_2.Target)

# Get best parameters
print("Best Hyperparameters:", grid_search.best_params_)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinu

Best Hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


In [7]:
data_2.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Month', 'Year',
       'SMA', 'EWMA_100', 'MiddleBand', 'UpperBand', 'LowerBand', 'RSI',
       'ForceIndex', 'EMV', 'Final_sentiment_Max_Max',
       'Final_sentiment_Max_Min', 'Final_sentiment_Max_Mean', 'India_Holidays',
       'Exchange_holidays_SETTLEMENT_HOLIDAY',
       'Exchange_holidays_TRADING_HOLIDAY', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Target'],
      dtype='object')

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Function to check directional accuracy and classification metrics
def calculate_metrics(actual, predicted):
    actual_direction = np.sign(np.diff(actual))  # 1 for up, -1 for down
    predicted_direction = np.sign(np.diff(predicted))

    correct_predictions = np.sum(actual_direction == predicted_direction)
    total_predictions = len(actual_direction)

    accuracy = correct_predictions / total_predictions * 100  # Percentage accuracy

    # Convert -1 labels to 0 for binary classification
    actual_direction = np.where(actual_direction == -1, 0, 1)
    predicted_direction = np.where(predicted_direction == -1, 0, 1)

    precision = precision_score(actual_direction, predicted_direction)
    recall = recall_score(actual_direction, predicted_direction)  # Added recall
    f1 = f1_score(actual_direction, predicted_direction)

    return accuracy, precision, recall, f1

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define feature set
sent_col = 'Final_sentiment_Max_Max'
metrics_df = pd.DataFrame(columns=["Sentiment Method", "Train MAPE", "Train RMSE", "Test MAPE", "Test RMSE", 'Train Directional Accuracy (%)', 'Test Directional Accuracy (%)',
       'Train Precision', 'Test Precision', 'Train Recall', 'Test Recall',
       'Train F1-score', 'Test F1-score'])
for sent_col in ['Final_sentiment_Max_Min', 'Final_sentiment_Max_Mean', 'Final_sentiment_Max_Max' ]:
    features = data_2[['Close', 'High', 'Low', 'Open', 'UpperBand', 'LowerBand', 'India_Holidays', 'Exchange_holidays_TRADING_HOLIDAY',\
                     'SMA', 'EWMA_100', 'Sunday', 'Monday',  'RSI']]
    
    target = data_2["Target"]
    
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)
    
    # Initialize and train the XGBoost model
    xgb_model = XGBRegressor(
        max_depth=3,
        learning_rate=0.1,  # Learning rate for boosting steps
        n_estimators=100,  # Number of trees
        colsample_bytree=0.8,  # Fraction of features to use for best splits
        subsample=0.8,  # Fraction of data used for training
        min_child_weight=1,
        objective='reg:squarederror',  # Regression objective
        random_state=42
    )
    
    xgb_model.fit(X_train, y_train)
    
    # Make predictions
    train_pred = xgb_model.predict(X_train)
    test_pred = xgb_model.predict(X_test)
    
    # Calculate evaluation metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    train_mape = np.mean(np.abs((y_train - train_pred) / y_train)) * 100
    test_mape = np.mean(np.abs((y_test - test_pred) / y_test)) * 100
    
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    # Calculate metrics for train and test sets
    train_accuracy, train_precision, train_recall, train_f1 = calculate_metrics(y_train, train_pred)
    test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(y_test, test_pred)
    
    metrics_data = {"Sentiment Method": sent_col,
        "Train RMSE": train_rmse,
        "Test RMSE": test_rmse,
        "Train MAPE": train_mape,
        "Test MAPE": test_mape,
        "Train R²": train_r2,
        "Test R²": test_r2,
        "Train Directional Accuracy (%)": train_accuracy,
        "Test Directional Accuracy (%)": test_accuracy,
        "Train Precision": train_precision,
        "Test Precision": test_precision,
        "Train Recall": train_recall,
        "Test Recall": test_recall,
        "Train F1-score": train_f1,
        "Test F1-score": test_f1
    }
    
    metrics_df = metrics_df.append(metrics_data, ignore_index=True)


/tmp/ipykernel_9148/1900784846.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(metrics_data, ignore_index=True)
/tmp/ipykernel_9148/1900784846.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(metrics_data, ignore_index=True)
/tmp/ipykernel_9148/1900784846.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(metrics_data, ignore_index=True)


In [16]:
metrics_df

,Sentiment Method,Train MAPE,Train RMSE,Test MAPE,Test RMSE,Train Directional Accuracy (%),Test Directional Accuracy (%),Train Precision,Test Precision,Train Recall,Test Recall,Train F1-score,Test F1-score,Train R²,Test R²
0,Final_sentiment_Max_Min,1.072951,24.323019,1.315205,32.358147,62.469734,56.310680,0.865297,0.747573,0.708411,0.611111,0.779034,0.672489,0.998298,0.855139
1,Final_sentiment_Max_Mean,1.068784,24.345620,1.341271,32.742686,62.469734,57.766990,0.858720,0.766990,0.727103,0.626984,0.787449,0.689956,0.998294,0.851675
2,Final_sentiment_Max_Max,1.063000,24.211848,1.372754,33.184199,61.501211,56.796117,0.872941,0.770833,0.693458,0.587302,0.772917,0.666667,0.998313,0.847648


In [18]:
metrics_df

,Sentiment Method,Train MAPE,Train RMSE,Test MAPE,Test RMSE,Train Directional Accuracy (%),Test Directional Accuracy (%),Train Precision,Test Precision,Train Recall,Test Recall,Train F1-score,Test F1-score,Train R²,Test R²
0,Final_sentiment_Max_Min,1.101416,24.882507,1.076414,29.425355,62.590799,56.31068,0.868664,0.762887,0.704673,0.587302,0.778122,0.663677,0.998218,0.880208
1,Final_sentiment_Max_Mean,1.101416,24.882507,1.076414,29.425355,62.590799,56.31068,0.868664,0.762887,0.704673,0.587302,0.778122,0.663677,0.998218,0.880208
2,Final_sentiment_Max_Max,1.101416,24.882507,1.076414,29.425355,62.590799,56.31068,0.868664,0.762887,0.704673,0.587302,0.778122,0.663677,0.998218,0.880208


In [73]:
# Set B
metrics_data = {
    "Train RMSE": [train_rmse],
    "Test RMSE": [test_rmse],
    "Train MAPE": [train_mape],
    "Test MAPE": [test_mape],
    "Train R²": [train_r2],
    "Test R²": [test_r2]
}

results_df = pd.DataFrame(metrics_data)
results_df

,Train RMSE,Test RMSE,Train MAPE,Test MAPE,Train R²,Test R²
0,28.849483,29.041217,1.66144,0.994022,0.997605,0.883315


In [100]:
results_df.columns

Index(['Train Directional Accuracy (%)', 'Test Directional Accuracy (%)',
       'Train Precision', 'Test Precision', 'Train Recall', 'Test Recall',
       'Train F1-score', 'Test F1-score'],
      dtype='object')

In [71]:
# Set C
metrics_data = {
    "Train RMSE": [train_rmse],
    "Test RMSE": [test_rmse],
    "Train MAPE": [train_mape],
    "Test MAPE": [test_mape],
    "Train R²": [train_r2],
    "Test R²": [test_r2]
}

results_df = pd.DataFrame(metrics_data)
results_df


,Train RMSE,Test RMSE,Train MAPE,Test MAPE,Train R²,Test R²
0,28.938395,29.325162,1.656079,0.984223,0.99759,0.881022


In [69]:
# Set D
metrics_data = {
    "Train RMSE": [train_rmse],
    "Test RMSE": [test_rmse],
    "Train MAPE": [train_mape],
    "Test MAPE": [test_mape],
    "Train R²": [train_r2],
    "Test R²": [test_r2]
}

results_df = pd.DataFrame(metrics_data)
results_df



,Train RMSE,Test RMSE,Train MAPE,Test MAPE,Train R²,Test R²
0,28.517171,30.661875,1.641388,1.092847,0.99766,0.869929


In [67]:
# Set E
metrics_data = {
    "Train RMSE": [train_rmse],
    "Test RMSE": [test_rmse],
    "Train MAPE": [train_mape],
    "Test MAPE": [test_mape],
    "Train R²": [train_r2],
    "Test R²": [test_r2]
}

results_df = pd.DataFrame(metrics_data)
results_df


,Train RMSE,Test RMSE,Train MAPE,Test MAPE,Train R²,Test R²
0,28.492797,31.261139,1.639556,1.13744,0.997664,0.864795


Train Directional Accuracy: 65.25%
Train Precision: 0.88
Train F1-score: 0.81

Test Directional Accuracy: 57.28%
Test Precision: 0.79
Test F1-score: 0.67


,Train Directional Accuracy (%),Test Directional Accuracy (%),Train Precision,Test Precision,Train F1-score,Test F1-score
0,65.254237,57.281553,0.878788,0.787234,0.814443,0.672727


In [ ]:
Train Directional Accuracy: 66.10%
Train Precision: 0.96
Train F1-score: 0.85

Test Directional Accuracy: 50.49%
Test Precision: 0.71
Test F1-score: 0.64